In [ ]:
import pandas as pd
import numpy as np
from shapely import Point, LineString
import geopandas as gpd
import gurobipy as gp

In [ ]:
def Subset(elements, types):
    return elements[elements["type"].isin(types)]

In [ ]:
raw_elements = pd.read_excel('../Data/Elements.xlsx')
from shapely import wkt

raw_elements['coor'] = raw_elements['coor'].apply(wkt.loads)
elements = gpd.GeoDataFrame(raw_elements, crs='epsg:4326', geometry='coor')
elements["length"] = elements.length

In [ ]:
elements = elements.fillna(0)
elements.plot(column="terminal")

In [ ]:
from collections import defaultdict

class Path:
    def __init__(self, elements):
        self.elements = elements

    def __eq__(self, other):
        if len(self.elements) != len(other.elements):
            return False
        return all(self.elements[i].name == other.elements[i].name for i in range(len(self.elements)))
    def __len__(self):
        return len(self.elements)

    def get_length(self):
        return sum(element.length for element in self.elements)

    def get_names(self):
        return ", ".join(e.name for e in self.elements)

    def get_elements_by_name(self):
        return [e.name for e in self.elements]
    def get_elements(self):
        return self.elements

class Element():
    """An element class to represent a node for A* Pathfinding"""

    def __init__(self, parent=None, element=None, g=1):
        self.parent = parent
        self.name = element["name"]
        self.position = element['coor']
        self.type = element['type']
        self.length = element['length']

        self.g = g
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.name == other.name

    def distance(self, other):
        return self.position.distance(other.position)

    def connectable(self, other, D):
        return self.name != other.name and self.position.distance(other["coor"]) < D #and self.type == other["type"]

def astar(elements, element_start, num_paths, D=100):
    """Returns a list of tuples as a path from the given start to the given end in the given maze"""

    # Create start and end node
    start_node = Element(None, element_start)
    start_node.g = start_node.h = start_node.f = 0
    end_element = elements[elements['name']==element_start["terminal"]]
    end_node = Element(None, end_element.squeeze() )
    end_node.g = end_node.h = end_node.f = 0

    next_g = 2
    g_multiplier = 2

    paths = []
    g_weights = {}
    possible_connections = pd.concat([Subset(elements, ["line"]), end_element], ignore_index=True)
    for c in range(num_paths):
        # Initialize both open and closed list
        open_list = []
        closed_list = []

        # Add the start node
        open_list.append(start_node)
        # Loop until you find the end
        while len(open_list) > 0:

            # Get the current node
            current_node = open_list[0]
            current_index = 0
            for index, item in enumerate(open_list):
                if item.f < current_node.f:
                    current_node = item
                    current_index = index

            # Pop current off open list, add to closed list
            open_list.pop(current_index)
            closed_list.append(current_node)

            # Found the goal
            if current_node == end_node:
                path = []
                current = current_node
                while current is not None:
                    path.append(current)
                    current = current.parent
                # print(g_weights)

                p = Path(path[::-1])
                if(p in paths or len(path)<=2):
                    # return paths
                    continue
                else:
                    paths.append(p) # Return reversed path
                    for e in p.get_elements():
                        if(e.name in g_weights.keys()):
                            g_weights[e.name] *= g_multiplier
                        else:
                            g_weights[e.name] = next_g
                    # print(p.get_names())
                    # print(g_weights)
                    break

            # Generate children
            connections = []
            for _,element in possible_connections.iterrows():
                already_visited = any(e.name == element["name"] for e in closed_list)
                if(already_visited):
                    continue
                if not current_node.connectable(element, D):
                    continue

                g = g_weights[element["name"]] if element["name"] in g_weights.keys() else next_g
                # Create new node
                new_node = Element(current_node, element, g)

                # Append
                connections.append(new_node)

            # Loop through connections
            for connection in connections:

                # Child is on the closed list
                for closed_child in closed_list:
                    if connection == closed_child:
                        continue

                # Create the f, g, and h values
                d = current_node.distance(connection)
                connection.g = connection.g + current_node.g + d
                connection.h = current_node.distance(end_node)
                connection.f = connection.g + connection.h
                # print(current_node.name, connection.name, connection.g, connection.h, connection.f)

                # Child is already in the open list
                for open_node in open_list:
                    if connection == open_node and connection.g > open_node.g:
                        continue

                # Add the child to the open list
                open_list.append(connection)
    return paths

In [ ]:
paths = []
for _, elem in elements[elements["type"]=="customer"].iterrows():
    p = astar(elements, elem.squeeze(), num_paths=4, D=20)
    paths.extend(p)

for p in paths:
    print(p.get_names())
print(f"Found {len(paths)} paths")

In [ ]:
distance_matrix = pd.DataFrame(index=elements['name'].values, columns=elements['name'].values, dtype=np.float64)

# Calculate distances
for i, geom1 in elements.iterrows():
    for j, geom2 in elements.iterrows():
        distance_matrix.at[geom1['name'], geom2['name']] = geom1['coor'].distance(geom2['coor'])

import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize = (16,6))
sns.heatmap(distance_matrix, annot=True)


In [ ]:
def BuildHMatrix(h_paths, elements):
    cols = elements["name"].values
    H = pd.DataFrame(columns=cols)

    for i,h in enumerate(h_paths):
        matrix_path = [0] * len(cols)
        for e in h.get_elements_by_name():
            idx = int(e[1:])
            matrix_path[idx-1] = 1
        H.loc[f"h{i+1}"] = matrix_path
    return H

In [ ]:
H = BuildHMatrix(paths, elements)

In [ ]:
C = Subset(elements, ["customer"])
T = Subset(elements, ["junction"])
R = Subset(elements, ["line"])
def MatrixDivision(H, C,R,T):
    Hc = H.iloc[:,  0               :len(C)]
    Hr = H.iloc[:,  len(C)          :len(C)+len(R)]
    Ht = H.iloc[:,  len(C)+len(R)   :-len(Subset(elements, ["transformer"]))]
    return Hc, Hr, Ht

In [ ]:
Hc, Hr, Ht = MatrixDivision(H, C,R,T)
Hc, Hr, Ht = Hc.values, Hr.values, Ht.values

# Optimization problem

In [ ]:
model = gp.Model()

MatrixTr_size = (len(T), len(R))
MatrixP_size = len(H)

# Decision Variables
Tr = model.addMVar(MatrixTr_size, vtype=gp.GRB.BINARY, name="Tr")
P = model.addMVar(MatrixP_size, vtype=gp.GRB.BINARY, name="P")

# Objective function 22a
model.setObjective( 10 * P.sum() - Tr.sum(), sense=gp.GRB.MAXIMIZE )

#Constaint 22b
for k in range(Tr.shape[0]):
    for m in range(MatrixP_size):
        model.addConstr( np.sum(Hr,axis=1)[m] * P[m] * np.transpose(Ht)[k,m] <= ( (Tr @ np.transpose(Hr)) * np.transpose(Ht))[k,m] )

#Constaint 22c
# for k in range(len(C)):
model.addConstr( P @ Hc <= 1)

#Constaint 22d
for k in range(len(R)):
    model.addConstr( gp.quicksum(Tr)[k] <= 1)

model.optimize()

In [ ]:
p_sol = []
for p in P:
    p_sol.append(p.X)
P_sol = pd.DataFrame(p_sol, dtype=np.int8).T
P_sol.columns = list(H.index)

Tr_sol = pd.DataFrame()
for tr in Tr:
    tr_sol = []
    for r  in tr:
        tr_sol.append(int(r.X))
    Tr_sol = pd.concat([Tr_sol, pd.DataFrame(tr_sol)], axis=1, ignore_index=True)
Tr_sol = Tr_sol.T
Tr_sol.columns = list(R["name"].values)
Tr_sol.index = list(T["name"].values)

# Results explanation

## P & Tr

In [ ]:
P_sol

In [ ]:
estimated_optimal_paths = []
for p in P_sol.columns.values:
    if P_sol[p][0] == 1:
        estimated_optimal_paths.append(p)
estimated_optimal_paths

In [ ]:
H

In [ ]:
Tr_sol

## First constrain


### left-hand side

In [ ]:
np.sum(Hr,axis=1)

In [ ]:
np.sum(Hr,axis=1) * P_sol.values

In [ ]:
np.sum(Hr,axis=1) * P_sol.values * np.transpose(Ht)

### rigth-hand side

In [ ]:
(Tr_sol.values @ np.transpose(Hr))

In [ ]:
(Tr_sol.values @ np.transpose(Hr)) * np.transpose(Ht) * P_sol.values

## Second constraint

In [ ]:
P_sol.values @ Hc

## Third constraint

In [ ]:
sum(Tr_sol.values)